# DURATION-NEUTRAL DTS-TARGET PORTFOLIO OPTIMIZATION

In [16]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Mean-Variance Optimizer
def mean_variance_optimizer(portfolio, target_dts_value, risk_free_rate):
    """
    Optimize portfolio weights to maximize weighted excess return while meeting target DTS.

    Parameters:
    - portfolio: pd.DataFrame
        Includes columns: 'DTS', 'ytm'
    - target_dts_value: float
    - risk_free_rate: float

    Returns:
    - np.ndarray: Optimized weights if successful, else None.
    """
    n = len(portfolio)

    def objective(weights):
        excess_returns = portfolio['ytm'] - risk_free_rate
        return -np.dot(weights, excess_returns)

    constraints = [
        {'type': 'eq', 'fun': lambda w: np.dot(w, portfolio['DTS']) - target_dts_value},
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    ]
    bounds = [(0, 1) for _ in range(n)]
    initial_weights = np.ones(n) / n

    result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)
    return result.x if result.success else None

# Hedging Function
def hedge_duration(portfolio_duration, treasury_data):
    """
    Hedge portfolio duration using Treasury instruments.

    Parameters:
    - portfolio_duration: float
    - treasury_data: pd.DataFrame

    Returns:
    - pd.DataFrame: Details of the Treasury instruments used for hedging.
    """
    remaining_duration = portfolio_duration
    hedge_results = []

    treasury_data = treasury_data.sort_values(by='treasury_duration', ascending=False)

    for _, treasury in treasury_data.iterrows():
        tool_duration = treasury['treasury_duration']
        max_quantity = treasury['available_quantity']

        needed_quantity = min(abs(remaining_duration / tool_duration), max_quantity)
        remaining_duration -= needed_quantity * tool_duration

        hedge_results.append({
            'treasury_tool': treasury.name,
            'used_quantity': needed_quantity,
            'cost': needed_quantity * treasury['price']
        })

        if abs(remaining_duration) < 1e-6:
            break

    return pd.DataFrame(hedge_results)

# Portfolio Return Calculation
def calculate_portfolio_metrics(sub_data, risk_free_rate, treasury_hedge):
    """
    Calculate portfolio returns including coupon income, capital gains, and hedging costs.

    Parameters:
    - sub_data: pd.DataFrame
    - risk_free_rate: float
    - treasury_hedge: pd.DataFrame

    Returns:
    - dict: Portfolio metrics and Sharpe Ratio.
    """
    coupon_income = (sub_data['Weight'] * sub_data['coupon_rate'] * 100).sum()
    capital_gains = (sub_data['Weight'] * (sub_data['price'] - sub_data['prev_price'])).sum()
    hedge_cost = treasury_hedge['cost'].sum() if not treasury_hedge.empty else 0
    portfolio_cost = (sub_data['Weight'] * sub_data['price']).sum()

    total_return = coupon_income + capital_gains - hedge_cost
    excess_return = total_return - risk_free_rate * portfolio_cost
    portfolio_return = total_return / portfolio_cost

    return {
        'coupon_income': coupon_income,
        'capital_gains': capital_gains,
        'hedge_cost': hedge_cost,
        'portfolio_cost': portfolio_cost,
        'total_return': total_return,
        'excess_return': excess_return,
        'portfolio_return': portfolio_return
    }

# Trading and Hedging Framework
def trading_and_hedging(trading_data, treasury_data, interest_rate_data, target_dts):
    """
    Perform trading and hedging operations, calculate portfolio metrics dynamically.

    Parameters:
    - trading_data: pd.DataFrame
    - treasury_data: pd.DataFrame
    - interest_rate_data: pd.DataFrame
    - target_dts: dict

    Returns:
    - pd.DataFrame: Results of trading, hedging, and performance metrics including Sharpe Ratio.
    """
    results = []
    sharpe_data = []

    trading_data = trading_data.sort_values(by=['CUSIP', 'Date'])
    trading_data['prev_price'] = trading_data.groupby('CUSIP')['price'].shift(1)

    for date, daily_data in trading_data.groupby('Date'):
        risk_free_rate = interest_rate_data.loc[interest_rate_data['Date'] == date, 'rate'].iloc[0]
        daily_treasury_data = treasury_data[treasury_data['Date'] == date]

        for subportfolio, sub_data in daily_data.groupby('subportfolio'):
            target_dts_value = target_dts.get(subportfolio)
            if target_dts_value is None or daily_treasury_data.empty:
                continue

            optimized_weights = mean_variance_optimizer(sub_data[['DTS', 'ytm']], target_dts_value, risk_free_rate)
            if optimized_weights is None:
                continue

            sub_data['Weight'] = optimized_weights
            portfolio_duration = (sub_data['Weight'] * sub_data['modified_duration']).sum()
            treasury_hedge = hedge_duration(portfolio_duration, daily_treasury_data)

            metrics = calculate_portfolio_metrics(sub_data, risk_free_rate, treasury_hedge)
            sharpe_data.append({'date': date, 'subportfolio': subportfolio, 'return': metrics['portfolio_return']})
            results.append({'date': date, 'subportfolio': subportfolio, **metrics})

    sharpe_ratios = []
    sharpe_df = pd.DataFrame(sharpe_data)

    for subportfolio, data in sharpe_df.groupby('subportfolio'):
        avg_return = data['return'].mean()
        return_volatility = data['return'].std()
        sharpe_ratio = (avg_return - risk_free_rate) / return_volatility if return_volatility > 0 else np.nan
        sharpe_ratios.append({'subportfolio': subportfolio, 'sharpe_ratio': sharpe_ratio})

    results_df = pd.DataFrame(results)
    sharpe_ratios_df = pd.DataFrame(sharpe_ratios)

    return results_df.merge(sharpe_ratios_df, on='subportfolio', how='left')

# Main Script
if __name__ == "__main__":
    
    """Input Data Structure:
    
    1. Trading Data (`trade.csv`):
       # I need trading duration + 1 pervious day to calculate return/vol
       
       - 'Date': str or datetime, trading date (e.g., '2023-01-01').
       - 'CUSIP': str, unique identifier for bonds 
       - 'DTS': float, Duration Times Spread of the bond.
       - 'ytm': float, yield-to-maturity of the bond (as a decimal, e.g., 0.05 for 5%).
       - 'price': float, current price of the bond.
       - 'coupon_rate': float, coupon rate of the bond (as a decimal, e.g., 0.04 for 4%).
       - 'modified_duration': float, modified duration of the bond.
       - 'subportfolio': str, category of the bond (e.g., 'high', 'medium', 'low').

    2. Treasury Data (`trea.csv`):
       - 'Date': str or datetime, date of the Treasury data (e.g., '2023-01-01').
       - 'treasury_duration': float, duration of the Treasury instrument.
       - 'price': float, price of the Treasury instrument.
       - 'available_quantity': float, quantity of the Treasury instrument available for hedging.

    3. Interest Rate Data (`interest.csv`):
       - 'Date': str or datetime, date of the interest rate (e.g., '2023-01-01').
       - 'rate': float, risk-free interest rate (as a decimal, e.g., 0.03 for 3%).

    4. Target DTS (`target_dts.csv` or dictionary in code):
       - Dictionary where keys are subportfolio names (e.g., 'high', 'medium', 'low').
       - Values are float, target DTS for the respective subportfolio.
    """
    
    trading_data = pd.read_csv("trade.csv")
    treasury_data = pd.read_csv("trea.csv")
    interest_rate_data = pd.read_csv("interest.csv")
    target_dts = {
        'high': ,
        'medium': ,
        'low': 
    }

    results = trading_and_hedging(trading_data, treasury_data, interest_rate_data, target_dts)

    print("Trading and Hedging Results with Sharpe Ratios:")
    print(results)


# DTS-NEUTRAL DURATION-TARGET PORTFOLIO OPTIMIZATION

In [ ]:
def dts_neutral_duration_target(trading_data, treasury_data, interest_rate_data, target_duration, cdx_data):
    """
    Construct a DTS-neutral portfolio with a target duration using CDX instruments.

    Parameters:
    - trading_data: pd.DataFrame
    - treasury_data: pd.DataFrame
    - interest_rate_data: pd.DataFrame
    - target_duration: dict
        Target modified duration values for subportfolios.
    - cdx_data: pd.DataFrame
        Includes columns: 'Date', 'cdx_dts', 'cdx_price', 'available_quantity'

    Returns:
    - pd.DataFrame: Results of trading, hedging, and performance metrics.
    """
    results = []
    sharpe_data = []

    trading_data = trading_data.sort_values(by=['CUSIP', 'Date'])
    trading_data['prev_price'] = trading_data.groupby('CUSIP')['price'].shift(1)

    for date, daily_data in trading_data.groupby('Date'):
        risk_free_rate = interest_rate_data.loc[interest_rate_data['Date'] == date, 'rate'].iloc[0]
        daily_treasury_data = treasury_data[treasury_data['Date'] == date]
        daily_cdx_data = cdx_data[cdx_data['Date'] == date]

        for subportfolio, sub_data in daily_data.groupby('subportfolio'):
            target_duration_value = target_duration.get(subportfolio)
            if target_duration_value is None or daily_treasury_data.empty or daily_cdx_data.empty:
                continue

            # Optimize portfolio weights
            optimized_weights = mean_variance_optimizer(sub_data[['modified_duration', 'ytm']], target_duration_value, risk_free_rate)
            if optimized_weights is None:
                continue

            sub_data['Weight'] = optimized_weights

            # Calculate duration-neutral DTS adjustments using CDX
            portfolio_dts = (sub_data['Weight'] * sub_data['DTS']).sum()
            cdx_dts = daily_cdx_data['cdx_dts'].iloc[0]  # Assuming one CDX value
            cdx_quantity = -portfolio_dts / cdx_dts  # Neutralize DTS
            cdx_cost = cdx_quantity * daily_cdx_data['cdx_price'].iloc[0]

            # Calculate portfolio duration for hedging
            portfolio_duration = (sub_data['Weight'] * sub_data['modified_duration']).sum()
            treasury_hedge = hedge_duration(portfolio_duration, daily_treasury_data)

            # Calculate portfolio returns
            metrics = calculate_portfolio_metrics(sub_data, risk_free_rate, treasury_hedge)
            metrics['cdx_cost'] = cdx_cost
            sharpe_data.append({'date': date, 'subportfolio': subportfolio, 'return': metrics['portfolio_return']})

            results.append({'date': date, 'subportfolio': subportfolio, **metrics})

    # Calculate Sharpe Ratios
    sharpe_ratios = []
    sharpe_df = pd.DataFrame(sharpe_data)

    for subportfolio, data in sharpe_df.groupby('subportfolio'):
        avg_return = data['return'].mean()
        return_volatility = data['return'].std()
        sharpe_ratio = (avg_return - risk_free_rate) / return_volatility if return_volatility > 0 else np.nan
        sharpe_ratios.append({'subportfolio': subportfolio, 'sharpe_ratio': sharpe_ratio})

    results_df = pd.DataFrame(results)
    sharpe_ratios_df = pd.DataFrame(sharpe_ratios)

    return results_df.merge(sharpe_ratios_df, on='subportfolio', how='left')

# Main Script
if __name__ == "__main__":
    """
    Additional Input Data for DTS-Neutral Portfolio:

    1. CDX Data (`cdx.csv`):
       - 'Date': str or datetime, date of the CDX data (e.g., '2023-01-01').
       - 'cdx_dts': float, Duration Times Spread of the CDX instrument.
       - 'cdx_price': float, price of the CDX instrument.
       - 'available_quantity': float, quantity of CDX instruments available for trading.

    Target Duration:
       - Dictionary where keys are subportfolio names (e.g., 'high', 'medium', 'low').
       - Values are float, target modified duration for the respective subportfolio.
    """
    # Load input data
    trading_data = pd.read_csv("trade.csv")  
    treasury_data = pd.read_csv("trea.csv")  
    interest_rate_data = pd.read_csv("interest.csv") 
    cdx_data = pd.read_csv("cdx.csv")  
    
    # Define Target Modified Duration
    target_duration = {
        'high': ,
        'medium': ,
        'low': 
    }

    results = dts_neutral_duration_target(trading_data, treasury_data, interest_rate_data, target_duration, cdx_data)

    print("DTS-Neutral Duration-Target Portfolio Results with Sharpe Ratios:")
    print(results)